In [1]:
import pandas as pd


In [85]:
df=pd.read_csv("/content/data.csv")
df.head(10)

,color,target
0,yellow,0
1,yellow,1
2,blue,1
3,yellow,1
4,red,1
5,yellow,0
6,red,1
7,red,1
8,yellow,1
9,blue,0


##One Hot Encoding

In [5]:
df1=pd.get_dummies(df,drop_first=True)
df1.head()

,target,color_red,color_yellow
0,0,0,1
1,1,0,1
2,1,0,0
3,1,0,1
4,1,1,0


In [7]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
one=OneHotEncoder()
enc=one.fit_transform(df).toarray()
df2=pd.DataFrame(enc)
df2.head()

,0,1,2,3,4
0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0


##Label Encoding

In [20]:
df['color']=df['color'].astype('category')
df['color']=df['color'].cat.codes

In [18]:
df['color']

0    2
1    2
2    0
3    2
4    1
5    2
6    1
7    1
8    2
9    0
Name: color, dtype: int8

In [21]:
from sklearn.preprocessing import LabelEncoder

In [32]:
le=LabelEncoder()
enc=le.fit_transform(df['color'])
df1=pd.DataFrame(enc,columns=['color1'])
pd.concat([df,df1],axis=1)
df.head()

,color,target,color1
0,2,0,2
1,2,1,2
2,0,1,0
3,2,1,2
4,1,1,1


##Count frequency

In [36]:
d1=pd.read_csv("data.csv")
d1.head()

,color,target
0,yellow,0
1,yellow,1
2,blue,1
3,yellow,1
4,red,1


In [39]:
enc=d1['color'].value_counts().to_dict()

In [43]:
d1['color1']=d1['color'].map(enc)

In [44]:
d1.head()

,color,target,color1
0,yellow,0,5
1,yellow,1,5
2,blue,1,2
3,yellow,1,5
4,red,1,3


In [45]:
fq=d1.groupby('color').size()/len(d1)
d1['color2']=d1['color'].map(fq)
d1.head()

,color,target,color1,color2
0,yellow,0,5,0.5
1,yellow,1,5,0.5
2,blue,1,2,0.2
3,yellow,1,5,0.5
4,red,1,3,0.3


#Ordinal Encoding

In [59]:
from sklearn.preprocessing import OrdinalEncoder

In [62]:

lo=OrdinalEncoder()
enc=lo.fit_transform(d1[['color']])
d2=pd.DataFrame(enc,columns=['color1'])
pd.concat([d1,d2],axis=1)
df.head()

,color,target,color1
0,2,0,2
1,2,1,2
2,0,1,0
3,2,1,2
4,1,1,1


##Mean Encoding

In [69]:
enc=d1.groupby(['color'])['target'].mean().to_dict()
d1['color3']=d1['color'].map(enc)
d1.head(12)

,color,target,color1,color2,color3
0,yellow,0,5,0.5,0.6
1,yellow,1,5,0.5,0.6
2,blue,1,2,0.2,0.5
3,yellow,1,5,0.5,0.6
4,red,1,3,0.3,1.0
5,yellow,0,5,0.5,0.6
6,red,1,3,0.3,1.0
7,red,1,3,0.3,1.0
8,yellow,1,5,0.5,0.6
9,blue,0,2,0.2,0.5


##Ordered Label Encoding

In [75]:
enc=d1.groupby(['color'])['target'].mean().sort_values().index
mapping={x:i for i,x in enumerate(enc,0)}
mapping


{'blue': 0, 'red': 2, 'yellow': 1}

In [76]:
d1['color4']=d1['color'].map(mapping)
d1.head()

,color,target,color1,color2,color3,color4
0,yellow,0,5,0.5,0.6,1
1,yellow,1,5,0.5,0.6,1
2,blue,1,2,0.2,0.5,0
3,yellow,1,5,0.5,0.6,1
4,red,1,3,0.3,1.0,2


##Weight Of Evidence
The rule is simple; WOE is the natural logarithm (ln) of the probability that the target equals 1 divided by the probability of the target equals 0.
Here is a mathematic formula : WOE = ln (p(1) / p(0)).
Where p(1) is the probability of the target being 1, and p(0) is the probability of the target being 0.

In [84]:
import numpy as np
dataframe = pd.DataFrame(d1.groupby(['color'])['target'].mean(),columns=['target'])
dataframe['non-target'] = 1 - dataframe['target']
dataframe['ratio'] = np.log(dataframe['target'] / dataframe['non-target'])
ratio_mapping = dataframe['ratio'].to_dict()
d1['color5'] = d1['color'].map(ratio_mapping)
dataframe.head()

,target,non-target,ratio
color,,,
blue,0.5,0.5,0.000000
red,1.0,0.0,inf
yellow,0.6,0.4,0.405465


In [83]:
d1.head(10)
ratio_mapping

{'blue': 0.0, 'red': inf, 'yellow': 0.4054651081081642}

##Probability Ratio Encoding
This encoding is suitable for classification problems only, where the target is binary.
It’s similar to WOE, but we don’t apply the natural logarithm. Instead, for each category, we calculate the mean of the target = 1, which is the probability of the target being 1 (P(1)), and the probability of the target being 0 (P(0)).
Finally, we calculate the ratio = P(1)/P(0), and we replace the categories by that ratio.



In [87]:
import numpy as np
dataframe = pd.DataFrame(d1.groupby(['color'])['target'].mean(),columns=['target'])
dataframe['non-target'] = 1 - dataframe['target']
dataframe['ratio'] = (dataframe['target'] / dataframe['non-target'])
ratio_mapping = dataframe['ratio'].to_dict()
d1['color6'] = d1['color'].map(ratio_mapping)
d1.head()
ratio_mapping

{'blue': 1.0, 'red': inf, 'yellow': 1.4999999999999998}